In [ ]:
def create_data_for_trigger_classification_network(cont_data,cont_labels,WL,margin,jump,end,start = cont_starting_time):
  N_len = len(cont_data)
  discrete_dataset = []
  discrete_labels = []
  location_labels = []

  i = start
  while(i + WL < end):
    count = 0
    loc_labs = []
    for j in range(margin,WL - margin):
      if cont_labels[i+j] != 0:
        count += 1
        loc_labs.append(j - margin)
    if count > 3:
      i += jump
      continue
    discrete_dataset.append(cont_data[i:i+WL])
    discrete_labels.append(count)
    location_labels.append(loc_labs)
    i += jump

  return discrete_dataset, discrete_labels, location_labels

def test_num_pulses_network(data_in,labels,end,prob=False):

  max_pulses = 3
  prior = np.ones(max_pulses + 1)
  count_all, count_MD, count_FA = 0,0,0

  if prob == True:
    num_windows = 0
    num_pulses = 0

    for i in range(cont_starting_time,end, jump):
      data = data_in[i:i+WL_trigger]
      labels1 = labels[i:i+WL_trigger]
      y_in = (data - norm_params_model_trigger_classification[0])/norm_params_model_trigger_classification[1]
      y_in = torch.Tensor(y_in)
      out = model_trigger_classification(y_in)
      out = out.detach().numpy()

      out_new = np.zeros(4)
      denum = np.exp(out[0]) + np.exp(out[1]) + np.exp(out[2]) + np.exp(out[3])
      for j in range(4):

        out_new[j] = np.exp(out[j])/denum

        out_new[j] = prior[j]*out_new[j]

      pred = np.argmax(out_new)

      num_windows += 1
      num_pulses += pred

    estimated_rate = num_pulses/(num_windows*1e-9*(WL_trigger - 2*margin))
    poisson_const = estimated_rate*1e-9*(WL_trigger - 2*margin)

    for j in range(4):
      prior[j] = (np.power(poisson_const,j)*np.exp(-poisson_const))/math.factorial(j)


  for i in range(cont_starting_time,end, jump):
    data = data_in[i:i+WL_trigger]
    labels1 = labels[i:i+WL_trigger]
    y_in = (data - norm_params_model_trigger_classification[0])/norm_params_model_trigger_classification[1]
    y_in = torch.Tensor(y_in)
    out = model_trigger_classification(y_in)
    out = out.detach().numpy()

    out_new = np.zeros(4)
    denum = np.exp(out[0]) + np.exp(out[1]) + np.exp(out[2]) + np.exp(out[3])
    for j in range(4):

      out_new[j] = np.exp(out[j])/denum

      out_new[j] = prior[j]*out_new[j]

    pred = np.argmax(out_new)

    real_out = 0
    for j in range(WL_trigger - 2*margin):
      if labels1[margin + j] != 0:
        real_out += 1

    count_all += real_out
    if real_out > pred:
      count_MD += real_out - pred
    if pred > real_out:
      count_FA += pred - real_out

  return count_all, count_MD, count_FA


def neural_networks_trigger(data_in, end, prob=False):

  max_pulses = 3
  prior = np.ones(max_pulses + 1)
  labels_out = np.zeros(len(data_in))

  if prob == True:

    num_windows = 0
    num_pulses = 0

    for i in range(0,len(data_in) - original_pulse_len, jump):
      data = data_in[i:i+WL_trigger]
      y_in = (data - norm_params_model_trigger_classification[0])/norm_params_model_trigger_classification[1]
      y_in = torch.Tensor(y_in)
      out = model_trigger_classification(y_in)
      out = out.detach().numpy()

      out_new = np.zeros(4)
      denum = np.exp(out[0]) + np.exp(out[1]) + np.exp(out[2]) + np.exp(out[3])
      for j in range(4):

        out_new[j] = np.exp(out[j])/denum

        out_new[j] = prior[j]*out_new[j]

      pred = np.argmax(out_new)

      num_windows += 1
      num_pulses += pred

    estimated_rate = num_pulses/(num_windows*1e-9*(WL_trigger - 2*margin))

    poisson_const = estimated_rate*1e-9*(WL_trigger - 2*margin)

    for j in range(4):
      prior[j] = (np.power(poisson_const,j)*np.exp(-poisson_const))/math.factorial(j)


  for i in range(cont_starting_time,end, jump):
    data = data_in[i:i+WL_trigger]
    y_in = (data - norm_params_model_trigger_classification[0])/norm_params_model_trigger_classification[1]
    y_in = torch.Tensor(y_in)
    out = model_trigger_classification(y_in)
    out = out.detach().numpy()

    out_new = np.zeros(4)
    denum = np.exp(out[0]) + np.exp(out[1]) + np.exp(out[2]) + np.exp(out[3])
    for j in range(4):

      out_new[j] = np.exp(out[j])/denum

      out_new[j] = prior[j]*out_new[j]

    pred = np.argmax(out_new)

    if pred == 1:
      y_in1 = (data - norm_params_model_arrival_times_1[0])/norm_params_model_arrival_times_1[1]
      y_in1 = torch.Tensor(y_in1)
      out1 = model_arrival_times_1(y_in1)
      out1 = int(torch.round(out1))

      labels_out[out1 + margin + i] += 1

    if pred == 2:

      y_in1 = (data - norm_params_model_arrival_times_2[0])/norm_params_model_arrival_times_2[1]
      y_in1 = torch.Tensor(y_in1)
      out1 = model_arrival_times_2(y_in1)
      out2 = int(torch.round(out1[0]))
      out3 = int(torch.round(out1[1]))
      labels_out[out2 + margin + i] += 1
      labels_out[out3 + margin + i] += 1

    if pred >= 3:

      y_in1 = (data - norm_params_model_arrival_times_3[0])/norm_params_model_arrival_times_3[1]
      y_in1 = torch.Tensor(y_in1)
      out1 = model_arrival_times_3(y_in1)
      out2 = int(torch.round(out1[0]))
      out3 = int(torch.round(out1[1]))
      out4 = int(torch.round(out1[2]))

      labels_out[out2 + margin + i] += 1
      labels_out[out3 + margin + i] += 1
      labels_out[out4 + margin + i] += 1

  return labels_out


def create_data_for_training_deep_trigger(mode,neutrons_curr,gammas_curr):

  data_0,data_1,data_2,data_3 = np.zeros((num_data,WL_trigger)),np.zeros((num_data,WL_trigger)),np.zeros((num_data,WL_trigger)),np.zeros((num_data,WL_trigger))
  labels_0,labels_1,labels_2,labels_3 = np.zeros(num_data),np.zeros(num_data),np.zeros(num_data),np.zeros(num_data)
  arr_times_1,arr_times_2,arr_times_3 = np.zeros(num_data),np.zeros((num_data,2)),np.zeros((num_data,3))
  count0,count1,count2,count3 = 0,0,0,0

  while(count0 < num_data or count1 < num_data or count2 < num_data or count3 < num_data):
    perm = np.random.permutation(np.shape(neutrons_curr)[0])
    neutrons_curr = neutrons_curr[perm]
    gammas_curr = gammas_curr[perm]

    noised_data,labels,true_inf,last_arrival_time = generate_pulse_train(neutrons_curr,gammas_curr,lambda_n,
                                                                                        lambda_g, return_statistics = False, add_noise = True)

    d_data, d_labels, d_loc_labels = create_data_for_trigger_classification_network(noised_data,labels, WL_trigger,margin, jump,last_arrival_time,start = cont_starting_time)

    for i in range(len(d_labels)):
      if d_labels[i] == 0 and count0 < num_data:
        data_0[count0,:] = d_data[i]
        labels_0[count0] = 0
        count0 += 1
      if d_labels[i] == 1 and count1 < num_data:
        data_1[count1,:] = d_data[i]
        labels_1[count1] = 1
        arr_times_1[count1] = d_loc_labels[i][0]
        count1 += 1
      if d_labels[i] == 2 and count2 < num_data:
        data_2[count2,:] = d_data[i]
        labels_2[count2] = 2
        arr_times_2[count2,:] = d_loc_labels[i][0:2]
        count2 += 1
      if d_labels[i] == 3 and count3 < num_data:
        data_3[count3,:] = d_data[i]
        labels_3[count3] = 3
        arr_times_3[count3,:] = d_loc_labels[i][0:3]
        count3 += 1

  np.save(curr_path + mode + '_DT_data_0',data_0)
  np.save(curr_path + mode + '_DT_labels_0',labels_0)

  np.save(curr_path + mode + '_DT_data_1',data_1)
  np.save(curr_path + mode + '_DT_labels_1',labels_1)
  np.save(curr_path + mode + '_DT_arr_times_1',arr_times_1)

  np.save(curr_path + mode + '_DT_data_2',data_2)
  np.save(curr_path + mode + '_DT_labels_2',labels_2)
  np.save(curr_path + mode + '_DT_arr_times_2',arr_times_2)

  np.save(curr_path + mode + '_DT_data_3',data_3)
  np.save(curr_path + mode + '_DT_labels_3',labels_3)
  np.save(curr_path + mode + '_DT_arr_times_3',arr_times_3)

def load_training_deep_trigger_data(mode):

  data_0 = np.load(curr_path + mode + '_DT_data_0.npy')
  labels_0 = np.load(curr_path + mode + '_DT_labels_0.npy')

  data_1 = np.load(curr_path + mode + '_DT_data_1.npy')
  labels_1 = np.load(curr_path + mode + '_DT_labels_1.npy')
  arr_times_1 = np.load(curr_path + mode + '_DT_arr_times_1.npy')

  data_2 = np.load(curr_path + mode + '_DT_data_2.npy')
  labels_2 = np.load(curr_path + mode + '_DT_labels_2.npy')
  arr_times_2 = np.load(curr_path + mode + '_DT_arr_times_2.npy')

  data_3 = np.load(curr_path + mode + '_DT_data_3.npy')
  labels_3 = np.load(curr_path + mode + '_DT_labels_3.npy')
  arr_times_3 = np.load(curr_path + mode + '_DT_arr_times_3.npy')

  return data_0,labels_0,data_1,labels_1,arr_times_1,data_2,labels_2,arr_times_2,data_3,labels_3,arr_times_3

def mix_data_for_model_num_pulses(data_0,data_1,data_2,data_3,labels_0,labels_1,labels_2,labels_3):

  data_mixed = np.concatenate((data_0,data_1,data_2,data_3),axis=0)
  labels_mixed = np.concatenate((labels_0,labels_1,labels_2,labels_3))
  perm = np.random.permutation(len(labels_mixed))
  out_data = data_mixed[perm]
  out_labels = labels_mixed[perm]

  return out_data, out_labels

def deep_trigger_arrival_times(data_in, end, prob=False):

  max_pulses = 3
  prior = np.ones(max_pulses + 1)
  labels_out = []

  if prob == True:

    num_windows = 0
    num_pulses = 0

    for i in range(0,len(data_in) - original_pulse_len, jump):
      data = data_in[i:i+WL_trigger]
      y_in = (data - norm_params_model_trigger_classification[0])/norm_params_model_trigger_classification[1]
      y_in = torch.Tensor(y_in)
      out = model_trigger_classification(y_in)
      out = out.detach().numpy()

      out_new = np.zeros(max_pulses + 1)
      denum = np.exp(out[0]) + np.exp(out[1]) + np.exp(out[2]) + np.exp(out[3])
      for j in range(max_pulses + 1):

        out_new[j] = np.exp(out[j])/denum

        out_new[j] = prior[j]*out_new[j]

      pred = np.argmax(out_new)

      num_windows += 1
      num_pulses += pred

    estimated_rate = num_pulses/(num_windows*1e-9*(WL_trigger - 2*margin))

    poisson_const = estimated_rate*1e-9*(WL_trigger - 2*margin)

    for j in range(max_pulses + 1):
      prior[j] = (np.power(poisson_const,j)*np.exp(-poisson_const))/math.factorial(j)

  else:
    if prob == False:
      prior = np.ones(max_pulses + 1)
    else:
      poisson_const = prob*1e-9*(WL_trigger - 2*margin)
      for j in range(max_pulses + 1):
        prior[j] = (np.power(poisson_const,j)*np.exp(-poisson_const))/math.factorial(j)
      prior[max_pulses] = 1 - sum(prior[:max_pulses])

  for i in range(cont_starting_time,end, jump):
    data = data_in[i:i+WL_trigger]
    y_in = (data - norm_params_model_trigger_classification[0])/norm_params_model_trigger_classification[1]
    y_in = torch.Tensor(y_in)
    out = model_trigger_classification(y_in)
    out = out.detach().numpy()

    out_new = np.zeros(4)
    denum = np.exp(out[0]) + np.exp(out[1]) + np.exp(out[2]) + np.exp(out[3])
    for j in range(4):

      out_new[j] = np.exp(out[j])/denum

      out_new[j] = prior[j]*out_new[j]

    pred = np.argmax(out_new)

    if pred == 1:
      y_in1 = (data - norm_params_model_arrival_times_1[0])/norm_params_model_arrival_times_1[1]
      y_in1 = torch.Tensor(y_in1)
      out1 = model_arrival_times_1(y_in1)
      out1 = int(torch.round(out1))

      labels_out.append(out1 + margin + i)

    if pred == 2:

      y_in1 = (data - norm_params_model_arrival_times_2[0])/norm_params_model_arrival_times_2[1]
      y_in1 = torch.Tensor(y_in1)
      out1 = model_arrival_times_2(y_in1)
      out2 = int(torch.round(out1[0]))
      out3 = int(torch.round(out1[1]))
      labels_out.append(out2 + margin + i)
      labels_out.append(out3 + margin + i)

    if pred >= 3:

      y_in1 = (data - norm_params_model_arrival_times_3[0])/norm_params_model_arrival_times_3[1]
      y_in1 = torch.Tensor(y_in1)
      out1 = model_arrival_times_3(y_in1)
      out2 = int(torch.round(out1[0]))
      out3 = int(torch.round(out1[1]))
      out4 = int(torch.round(out1[2]))

      labels_out.append(out2 + margin + i)
      labels_out.append(out3 + margin + i)
      labels_out.append(out4 + margin + i)

  return np.array(labels_out)

def deep_trigger_arrival_times_NM(data_in, end, prob=False):

  max_pulses = 3
  prior = np.ones(max_pulses + 1)
  labels_out = []

  if prob == True:

    num_windows = 0
    num_pulses = 0

    for i in range(0,len(data_in) - original_pulse_len, jump):
      data = data_in[i:i+WL_trigger_middle]
      y_in = (data - norm_params_model_trigger_classification_NM[0])/norm_params_model_trigger_classification_NM[1]
      y_in = torch.Tensor(y_in)
      out = model_trigger_classification_NM(y_in)
      out = out.detach().numpy()

      out_new = np.zeros(max_pulses + 1)
      denum = np.exp(out[0]) + np.exp(out[1]) + np.exp(out[2]) + np.exp(out[3])
      for j in range(max_pulses + 1):

        out_new[j] = np.exp(out[j])/denum

        out_new[j] = prior[j]*out_new[j]

      pred = np.argmax(out_new)

      num_windows += 1
      num_pulses += pred

    estimated_rate = num_pulses/(num_windows*1e-9*(WL_trigger_middle))

    poisson_const = estimated_rate*1e-9*(WL_trigger_middle)

    for j in range(max_pulses + 1):
      prior[j] = (np.power(poisson_const,j)*np.exp(-poisson_const))/math.factorial(j)

  else:
    if prob == False:
      prior = np.ones(max_pulses + 1)
    else:
      poisson_const = prob*1e-9*(WL_trigger_middle)
      for j in range(max_pulses + 1):
        prior[j] = (np.power(poisson_const,j)*np.exp(-poisson_const))/math.factorial(j)
      prior[max_pulses] = 1 - sum(prior[:max_pulses])



  for i in range(cont_starting_time,end, jump):
    data = data_in[i:i+WL_trigger_middle]
    y_in = (data - norm_params_model_trigger_classification_NM[0])/norm_params_model_trigger_classification_NM[1]
    y_in = torch.Tensor(y_in)
    out = model_trigger_classification_NM(y_in)
    out = out.detach().numpy()

    out_new = np.zeros(4)
    denum = np.exp(out[0]) + np.exp(out[1]) + np.exp(out[2]) + np.exp(out[3])
    for j in range(4):

      out_new[j] = np.exp(out[j])/denum

      out_new[j] = prior[j]*out_new[j]

    pred = np.argmax(out_new)

    if pred == 1:
      y_in1 = (data - norm_params_model_arrival_times_1_NM[0])/norm_params_model_arrival_times_1_NM[1]
      y_in1 = torch.Tensor(y_in1)
      out1 = model_arrival_times_1_NM(y_in1)
      out1 = int(torch.round(out1))

      labels_out.append(out1 + i)

    if pred == 2:

      y_in1 = (data - norm_params_model_arrival_times_2_NM[0])/norm_params_model_arrival_times_2_NM[1]
      y_in1 = torch.Tensor(y_in1)
      out1 = model_arrival_times_2_NM(y_in1)
      out2 = int(torch.round(out1[0]))
      out3 = int(torch.round(out1[1]))
      labels_out.append(out2 + i)
      labels_out.append(out3 + i)

    if pred >= 3:

      y_in1 = (data - norm_params_model_arrival_times_3_NM[0])/norm_params_model_arrival_times_3_NM[1]
      y_in1 = torch.Tensor(y_in1)
      out1 = model_arrival_times_3_NM(y_in1)
      out2 = int(torch.round(out1[0]))
      out3 = int(torch.round(out1[1]))
      out4 = int(torch.round(out1[2]))

      labels_out.append(out2 + i)
      labels_out.append(out3 + i)
      labels_out.append(out4 + i)

  return np.array(labels_out)

def create_data_for_training_deep_trigger_new(mode,neutrons_curr,gammas_curr):

  data_0,data_1,data_2,data_3 = np.zeros((num_data,WL_trigger)),np.zeros((num_data,WL_trigger)),np.zeros((num_data,WL_trigger)),np.zeros((num_data,WL_trigger))
  labels_0,labels_1,labels_2,labels_3 = np.zeros(num_data),np.zeros(num_data),np.zeros(num_data),np.zeros(num_data)
  arr_times_1,arr_times_2,arr_times_3 = np.zeros(num_data),np.zeros((num_data,2)),np.zeros((num_data,3))
  count0,count1,count2,count3 = 0,0,0,0

  while(count0 < num_data or count1 < num_data or count2 < num_data or count3 < num_data):
    perm = np.random.permutation(np.shape(neutrons_curr)[0])
    neutrons_curr = neutrons_curr[perm]
    gammas_curr = gammas_curr[perm]

    noised_data,labels,true_inf,last_arrival_time = generate_pulse_train(neutrons_curr,gammas_curr,lambda_n,
                                                                                        lambda_g, return_statistics = False, add_noise = True)

    d_data, d_labels, d_loc_labels = create_data_for_trigger_classification_network(noised_data,labels, WL_trigger,margin, jump,last_arrival_time,start = cont_starting_time)

    for i in range(len(d_labels)):
      if d_labels[i] == 0 and count0 < num_data:
        data_0[count0,:] = d_data[i]
        labels_0[count0] = 0
        count0 += 1
      if d_labels[i] == 1 and count1 < num_data:
        data_1[count1,:] = d_data[i]
        labels_1[count1] = 1
        arr_times_1[count1] = d_loc_labels[i][0]
        count1 += 1
      if d_labels[i] == 2 and count2 < num_data:
        data_2[count2,:] = d_data[i]
        labels_2[count2] = 2
        arr_times_2[count2,:] = d_loc_labels[i][0:2]
        count2 += 1
      if d_labels[i] == 3 and count3 < num_data:
        data_3[count3,:] = d_data[i]
        labels_3[count3] = 3
        arr_times_3[count3,:] = d_loc_labels[i][0:3]
        count3 += 1

  return data_0,labels_0,data_1,labels_1,arr_times_1,data_2,labels_2,arr_times_2,data_3,labels_3,arr_times_3